In [1]:
#Load the dataset 
import json
with open('/content/800-tweets-annotated.json', 'r') as f:
    data = json.load(f)

In [2]:
#Normalise the Json file into a dataframe
import pandas as pd
d=pd.json_normalize(data, record_path='data')

In [3]:
hh=pd.DataFrame(d)

In [4]:
#Exapand the annotated data into new dataframe
df1=hh.explode('examples').reset_index()

In [5]:
#A training data is created with tweet as text and tagged entities as label
training_data = []
for example in df1['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)
  
print(training_data)

[{'text': "RT @AlexandraUlmer: Now for the bad news.... Seismologist says Tuesday's 8.2 quake in #Chile was likely not 'the big one' http://t.co/wHwiX�??", 'entities': [(77, 82, 'TYPE OF IMPACT'), (83, 85, 'LOCATION MODIFIER'), (73, 82, 'SEVERITY OR QUANTITY'), (87, 92, 'PLACE NAME'), (73, 76, 'SEVERITY OR QUANTITY')]}, {'text': 'IG ; JeremyHumilde Gobierno de Chile trabaja para restablecer servicios tras el terremoto: (CNN Espa?�ol) �?? Des... http://t.co/K3vq2Owqly', 'entities': []}, {'text': 'RT @PFourDigital: A powerful earthquake hit Chile last night. Magnitude 8.2 that produced tsunamis that are already hitting the coast. #Fue�??', 'entities': [(44, 49, 'PLACE NAME'), (124, 133, 'LOCATION MODIFIER'), (29, 39, 'TYPE OF IMPACT'), (90, 98, 'SEVERITY OR QUANTITY'), (62, 75, 'SEVERITY OR QUANTITY'), (90, 98, 'TYPE OF IMPACT'), (128, 133, 'ITEM AFFECTED')]}, {'text': '@PatriciaLMalik The 7.8 magnitude #earthquake in #Pakistan creates island off the coast of Balochistan http://t.co/O6Y6

In [6]:
#split the dataset for training & evaluating
from sklearn.model_selection import train_test_split
train, test = train_test_split(training_data, test_size=0.3)


In [7]:
#Required imports for Spacy Architecture
import spacy
#import spacy_transformers
from spacy.tokens import DocBin
from tqdm import tqdm
doc_bin = DocBin()

Model 1: Spacy large module using English Language

In [8]:
#Download the large module of English language
!python -m spacy download en_core_web_lg 

2022-09-26 16:16:01.366582: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 8.7 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [13]:
#convert into doc object
from spacy.util import filter_spans
nlp = spacy.load("en_core_web_lg")
for training_example  in tqdm(train): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train-modelA.spacy")

 73%|███████▎  | 409/560 [00:00<00:00, 1380.11it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 560/560 [00:00<00:00, 1355.71it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [14]:
from spacy.util import filter_spans
nlp = spacy.load("en_core_web_lg")
for training_example  in tqdm(test): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("test-modelA.spacy")

100%|██████████| 240/240 [00:00<00:00, 1403.19it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


Create base_configuration_modelA file from the link given and Choose the below options:

1.   Language: English
2.   Components: ner
3.   Hardware:CPU
4.   Optimize for: Accuracy


https://spacy.io/usage/training#quickstart

In [11]:
!python -m spacy init fill-config base_config_modelA.cfg config_modelA.cfg

2022-09-26 16:17:53.321613: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config_modelA.cfg
You can now add your data and train your pipeline:
python -m spacy train config_modelA.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Change the following paramaters from config_modelA.cfg file:

1.   batch_size = 16
2.   dropout = 0.25
3.   max_epochs=100



In [15]:
!python -m spacy train config_modelA.cfg --output ./Output_A/ --paths.train ./train-modelA.spacy --paths.dev ./test-modelA.spacy

2022-09-26 16:21:30.357872: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: Output_A
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-09-26 16:21:31,206] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-09-26 16:21:31,221] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-09-26 16:21:31,227] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-09-26 16:21:33,367] [INFO] Added vectors: en_core_web_lg
INFO:spacy:Added vectors: en_core_web_lg
tcmalloc: large alloc 1233977344 bytes == 0x38ab8000 @  0x7f25527122a4 0x7f25473d9a7d 0x7f25473d8c5d 0x7f25473d5435 0x7f25473d5bde 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51837f 0x7f2463424218 0x7f24634261de 0x7f246342d0d7 0x7f24634307df 0x590055 0x51b180 0x58f2a7 0x51740e 0x7f2463424218 0x7f24634261de 0x7f246342cdb0

In [17]:
nlp_ner_ModelA = spacy.load("/content/Output_A/model-best")

In [20]:
#testing the model

doc = nlp_ner_ModelA(test[123]['text'])

colors = {"PLACE NAME": "#F67DE3", "LOCATION MODIFIER": "#7DF6D9", "ITEM AFFECTED":"#de6262", "TYPE OF IMPACT":"#a6e22d",
          "SEVERITY OR QUANTITY":"#cc2b5e"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)


In [22]:
!python -m spacy evaluate ./Output_A/model-best ./test-modelA.spacy --output ./output-modelA

2022-09-26 17:29:36.897040: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   98.43 
NER R   98.32 
NER F   98.38 
SPEED   2832  


=============================== NER (per type) ===============================

                           P       R       F
SEVERITY OR QUANTITY   97.31   98.91   98.10
TYPE OF IMPACT         99.04   98.64   98.84
LOCATION MODIFIER      97.73   99.74   98.72
PLACE NAME             99.13   98.15   98.64
ITEM AFFECTED          98.02   94.64   96.30

✔ Saved results to output-modelA


Model 2: Spacy transformer module using English Language

In [23]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 9.4 MB/s 
     |████████████████████████████████| 4.7 MB 37.8 MB/s 
     |████████████████████████████████| 6.6 MB 38.2 MB/s 
     |████████████████████████████████| 120 kB 53.6 MB/s 


In [24]:
import spacy_transformers

In [25]:
#Download the trasformer model
!python -m spacy download en_core_web_trf

2022-09-26 17:30:43.018379: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 33 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [26]:
from spacy.util import filter_spans
nlp = spacy.load("en_core_web_trf")
for training_example  in tqdm(train): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train-modelB.spacy")

 12%|█▎        | 70/560 [00:00<00:01, 363.26it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 31%|███       | 171/560 [00:00<00:00, 422.61it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 52%|█████▎    | 294/560 [00:00<00:00, 541.98it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 78%|███████▊  | 435/560 [00:00<00:00, 631.55it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity

100%|██████████| 560/560 [00:01<00:00, 545.88it/s]



Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [27]:
from spacy.util import filter_spans
nlp = spacy.load("en_core_web_trf")
for training_example  in tqdm(test): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("test-modelB.spacy")

100%|██████████| 240/240 [00:00<00:00, 1323.82it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


https://spacy.io/usage/training#quickstart

Create the base_config_modelB file with vectors = "en_core_web_trf"

In [29]:
!python -m spacy init fill-config base_config_modelB.cfg config_modelB.cfg

2022-09-26 17:34:10.362542: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config_modelB.cfg
You can now add your data and train your pipeline:
python -m spacy train config_modelB.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Change the following paramaters from config_modelA.cfg file:

1.   batch_size = 16
2.   dropout = 0.25
3.   max_epochs=100



In [30]:
!python -m spacy train config_modelB.cfg --output ./Output_B/ --paths.train ./train-modelB.spacy --paths.dev ./test-modelB.spacy


Aborted!


In [32]:
nlp_ner_ModelB = spacy.load("/content/Output_B/model-best")

In [34]:
#testing the model

doc = nlp_ner_ModelB(test[123]['text'])

colors = {"PLACE NAME": "#F67DE3", "LOCATION MODIFIER": "#7DF6D9", "ITEM AFFECTED":"#de6262", "TYPE OF IMPACT":"#a6e22d",
          "SEVERITY OR QUANTITY":"#cc2b5e"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)


In [35]:
!python -m spacy evaluate ./Output_B/model-best ./test-modelB.spacy --output ./output-modelB

2022-09-26 18:33:30.821639: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   97.91 
NER R   98.83 
NER F   98.37 
SPEED   2864  


=============================== NER (per type) ===============================

                           P       R       F
SEVERITY OR QUANTITY   97.49   99.09   98.28
TYPE OF IMPACT         98.12   99.46   98.78
LOCATION MODIFIER      99.22   98.45   98.84
PLACE NAME             97.80   98.89   98.34
ITEM AFFECTED          96.56   96.93   96.75

✔ Saved results to output-modelB
